In [1]:
import os
import torch
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/uj-user/Yo/hybrid-ltm/ltm-venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "/home/uj-user/Yo/hybrid-ltm/data-80/context-fms-train/MS_DG/test.jsonl"
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/last_session.jsonl"

# model_name = "/home/uj-user/Yo/hybrid-ltm/model/llama-7b_ALL"
model_name = "/home/uj-user/Yo/hybrid-ltm/model/llama-7b_MS_DG"
# model_name = "gpt2"
max_len = 2048
target_len = 128
device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.31it/s]


In [4]:
dataset = pd.read_json(data_path, lines=True)
dataset

,input
0,```You will be shown a 1 session dialogues bet...
1,```You will be shown a 1 session dialogues bet...
2,```You will be shown a 1 session dialogues bet...
3,```You will be shown a 1 session dialogues bet...
4,```You will be shown a 1 session dialogues bet...
...,...
75,```You will be shown a 1 session dialogues bet...
76,```You will be shown a 1 session dialogues bet...
77,```You will be shown a 1 session dialogues bet...
78,```You will be shown a 1 session dialogues bet...


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left', use_fast=False)
# tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'})
tokenizer

LlamaTokenizer(name_or_path='/home/uj-user/Yo/hybrid-ltm/model/llama-7b_MS_DG', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
dataset['input'][0]

"```You will be shown a 1 session dialogues between Patient and Doctor. Please read and understand given multiple Dialogue Session, then complete the task under the guidance of Task Introduction.\n\n\nDialogue Session #3:\nDoctor:Hello Patient, how can I assist you today?\nPatient:Hi Doctor, I've been thinking of traveling to a new destination.\nDoctor:That sounds like a great idea. Have you decided on where you want to go?\nPatient:I haven't decided yet, but I'm looking for recommendations.\nDoctor:Well, I recently went to Japan and it was a wonderful experience. The culture, food, and people were amazing.\nPatient:That sounds interesting. How was the travel experience for you?\nDoctor:It was a long flight, but I made sure to stay hydrated and take breaks to stretch. Have you traveled internationally before?\nPatient:Yes, a few years ago I went to Europe.\nDoctor:That's great. Did you have any issues with your health during that trip?\nPatient:No, everything was fine. Although, I did 

In [7]:
ans_jsons = []
for i, line in enumerate(tqdm(dataset['input'])):
    input = tokenizer([dataset['input'][i]], max_length=(max_len-target_len), truncation=True, return_tensors='pt').to(device)#.input_ids
    target_len = min(len(input.input_ids[0])+target_len, max_len)
    output_ids = model.generate(
        **input,
        do_sample=True,
        max_length=target_len,
        encoder_no_repeat_ngram_size=None
    )
    output_ids = output_ids[0][len(input.input_ids[0]):]
    outputs = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    # print('input:\n', tokenizer.decode(input.input_ids[0]))
    # print('output:\n', outputs)
    # if i%3==1:
    #     break
    # print(i)
    ans_jsons.append(
        {
            "question_id": i,
            "text": outputs,
            "model_id": model_name,
            "metadata": {},
        }
    )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [8]:
filename = f"predictions/{model_name}_{data_path.split('/')[-2]}.json"
save_path = os.path.dirname(filename)
os.makedirs(save_path, exist_ok=True)
df_ans_json = pd.DataFrame(ans_jsons)
df_ans_json.to_json(filename, orient='records', lines=True)

In [9]:
filename = f"predictions/{model_name}_{data_path.split('/')[-2]}_last.json"
save_path = os.path.dirname(filename)
os.makedirs(save_path, exist_ok=True)
df_ans_json = pd.DataFrame(ans_jsons)
df_ans_json.to_json(filename, orient='records', lines=True)